In [1]:
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information scipy, numpy, matplotlib, pandas, version_information
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

This notebook was generated at 2019-07-03 11:41:26 (KST = GMT+0900) 
0 Python     3.7.3 64bit [Clang 4.0.1 (tags/RELEASE_401/final)]
1 IPython    6.5.0
2 OS         Darwin 18.6.0 x86_64 i386 64bit
3 scipy      1.2.1
4 numpy      1.16.4
5 matplotlib 3.1.0
6 pandas     0.24.2
7 version_information 1.0.3


In [1]:
from pathlib import Path
from snuo1mpy import Preprocessor
import numpy as np
TOPDIR = Path(".")
RAWDIR = Path(".")
ARCHIVE = Path("archive")
CALIBDIR = Path("calib")

# At the current version of SNUO1Mpy, the following bias_kw and dark_kw are 
# identical to the defaults. I just explicitly wrote them for clarity.

bias_kw = dict(bias_type_key=["OBJECT"], bias_type_val=["bias"], bias_group_key=[])
dark_kw = dict(dark_type_key=["OBJECT"], dark_type_val=["dark"], dark_group_key=["EXPTIME"])
flat_kw = dict(flat_type_key=["IMAGETYP"], flat_type_val=["Flat Field"], flat_group_key=["FILTER"])

p = Preprocessor(topdir=TOPDIR, rawdir=RAWDIR,instrument="STX16803",
                 **bias_kw, **dark_kw, **flat_kw) 

In [4]:
p.organize_raw(archive_dir=ARCHIVE)

CCD Image 17.fit is not a regular name. Moving to useless. 
CCD Image 18.fit is not a regular name. Moving to useless. 
CCD Image 2.fit is not a regular name. Moving to useless. 


In [5]:
p.make_bias(savedir=CALIBDIR)

Analyzing FITS... Extracting keys:  ['OBJECT']
Done.
9 FITS files with "['OBJECT'] = ['bias']" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Writing FITS to calib/bias.fits... Saved.


In [6]:
p.make_dark(savedir=CALIBDIR)

Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 0.4]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using calib/bias.fits
Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 60.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using calib/bias.fits


In [7]:
p.make_flat(savedir=CALIBDIR)

/Users/ysbach/anaconda3/lib/python3.6/site-packages/snuo1mpy-0.0.1.dev0-py3.6.egg/snuo1mpy/preprocessor.py:596: UserWarning: Dark not available for ('dark', 0.1). Processing without dark.


Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105650-R-0.1_BD.fits... Saved.
Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105748-R-0.4_BD.fits... Saved.
Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105802-R-0.4_BD.fits... Saved.
Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105816-R-0.4_BD.fits... Saved.
Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105830-R-0.4_BD.fits... Saved.
Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105844-R-0.4_BD.fits... Saved.
Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105858-R-0.4_BD.fits... Saved.
Bias subtracted using calib/bias.fits
Writing FITS to flat/SNUO_STX16803-flat-1-1-20190602-105912-R-0.4_BD.fits... Saved.
Bias subtracted using ca

Now, Cosmic Ray rejection. 

Of course you should not use the CRREJ keys blindly below. 

I tested with some combinations and found ``sigfrac`` and ``objlim`` did not give much difference. Also the ``psf~~~`` parameters are not touched (i.e., they are just dummy keywords) because I used ``fsmode='median'`` (see astroscrappy doc). Also please note that  crrej should be done **AFTER** bias/dark and flat correction (see [van Dokkum's note](http://www.astro.yale.edu/dokkum/lacosmic/notes.html)).


You may try some simple codes like
```python
%matplotlib widget
from astroscrappy import detect_cosmics
import ysfitsutilpy as yfu
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams

plt.style.use('default')
rcParams.update({'font.size':12})

ccd = yfu.load_ccd(<BDF_Processed_FITS_path>)
hdr = ccd.header

crrej_kwargs = dict(sigclip=6, sigfrac=0.5, objlim=1.0,
                  satlevel=np.inf, pssl=0.0, niter=4, sepmed=True,
                  cleantype='medmask', fsmode='median', psfmodel='gauss',
                  psffwhm=2.5, psfsize=7, psfk=None, psfbeta=4.765)

crmask, cleanarr = detect_cosmics(ccd.data,
                                  gain=hdr["GAIN"],
                                  readnoise=hdr["RDNOISE"],
                                  **crrej_kwargs,
                                  verbose=True)

fig, axs = plt.subplots(1, 3, figsize=(15, 6), sharex=True, sharey=True)
yfu.zimshow(axs[0], ccd.data)
yfu.zimshow(axs[1], cleanarr / hdr["GAIN"])
axs[2].imshow(crmask, origin="lower")
plt.tight_layout()
```

> FYI, for jupyter lab to use matplotlib widget, see [here](https://github.com/matplotlib/jupyter-matplotlib)
> ```
> conda install -c conda-forge ipympl
> # If using the Notebook
> conda install -c conda-forge widgetsnbextension
> # If using JupyterLab
> conda install nodejs
> jupyter labextension install @jupyter-widgets/jupyterlab-manager
> jupyter labextension install jupyter-matplotlib
> jupyter nbextension enable --py widgetsnbextension
> ```

In [4]:
crrej_kwargs = dict(sigclip=6, sigfrac=0.5, objlim=1.0,
                    satlevel=np.inf, pssl=0.0, niter=4, sepmed=True,
                    cleantype='medmask', fsmode='median', psfmodel='gauss',
                   psffwhm=2.5, psfsize=7, psfk=None, psfbeta=4.765)
 
p.do_preproc(savedir="reduced", do_crrej=True, crrej_kwargs=crrej_kwargs, verbose_crrej=True)

Bias subtracted using calib/bias.fits
Dark subtracted using calib/dark-60.0.fits
Flat corrected using calib/Flat Field-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
331 cosmic pixels this iteration
Iteration 2:
190 cosmic pixels this iteration
Iteration 3:
207 cosmic pixels this iteration
Iteration 4:
228 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), LACOSMIC default setting: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-kw4-1-1-20190602-115534-R-60.0.fits... Saved.
Bias subtracted using calib/bias.fits
Dark subtracted using calib/dark-60.0.fits
Flat corrected using calib/Flat Field-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnois

,file,DATE-OBS,FILTER,OBJECT,EXPTIME,IMAGETYP,AIRMASS,XBINNING,YBINNING,CCD-TEMP,SET-TEMP,OBJCTRA,OBJCTDEC,OBJCTALT,PROCESS
0,reduced/SNUO_STX16803-kw4-1-1-20190602-115534-...,2019-06-02T11:55:34,R,kw4,60.0,Light Frame,None,1,1,-10.00000,-10.0,None,None,None,BDFC
1,reduced/SNUO_STX16803-kw4-1-1-20190602-115648-...,2019-06-02T11:56:48,R,kw4,60.0,Light Frame,None,1,1,-10.00000,-10.0,None,None,None,BDFC
2,reduced/SNUO_STX16803-kw4-1-1-20190602-115801-...,2019-06-02T11:58:01,R,kw4,60.0,Light Frame,None,1,1,-9.96875,-10.0,None,None,None,BDFC
3,reduced/SNUO_STX16803-kw4-1-1-20190602-115915-...,2019-06-02T11:59:15,R,kw4,60.0,Light Frame,None,1,1,-10.00000,-10.0,None,None,None,BDFC
4,reduced/SNUO_STX16803-kw4-1-1-20190602-120029-...,2019-06-02T12:00:29,R,kw4,60.0,Light Frame,None,1,1,-9.96875,-10.0,None,None,None,BDFC
5,reduced/SNUO_STX16803-kw4-1-1-20190602-120142-...,2019-06-02T12:01:42,R,kw4,60.0,Light Frame,None,1,1,-10.00000,-10.0,None,None,None,BDFC
6,reduced/SNUO_STX16803-kw4-1-1-20190602-120256-...,2019-06-02T12:02:56,R,kw4,60.0,Light Frame,None,1,1,-10.00000,-10.0,None,None,None,BDFC
7,reduced/SNUO_STX16803-kw4-1-1-20190602-120410-...,2019-06-02T12:04:10,R,kw4,60.0,Light Frame,None,1,1,-10.00000,-10.0,None,None,None,BDFC
8,reduced/SNUO_STX16803-kw4-1-1-20190602-120523-...,2019-06-02T12:05:23,R,kw4,60.0,Light Frame,None,1,1,-10.00000,-10.0,None,None,None,BDFC
9,reduced/SNUO_STX16803-kw4-1-1-20190602-120637-...,2019-06-02T12:06:37,R,kw4,60.0,Light Frame,None,1,1,-9.96875,-10.0,None,None,None,BDFC


In [2]:
p.make_astrometry_script(output=Path("astrometry.sh"))

In [6]:
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was finished at {now} ")

This notebook was finished at 2019-07-03 12:30:53 (KST = GMT+0900) 


If you have a machine with astrometry.net installed, make a cell and run
```
%%bash
script astrometry.log
bash astrometry.sh
exit
```